<h2>
    <p>
        MÔN DATA MINING 2019
    </p>
    <p style="color:red">
        LÊ NGUYỄN SƠN NGUYÊN - CH1702039 (NHÓM 9)
    </p>
    <P>
        ĐỀ TÀI: Phân tích quan điểm (Sentiment Analysis) cho CSDL Internet Movie Database (IMDb)
    </P>
</h2>

<h3>
    <p style="color:red">
        SỬ DỤNG SPARK VÀ BIGDL TRONG VIỆC TẠO MODEL PHỨC TẠP DẠNG KERAS CÓ KHẢ NĂNG TRAINING PHÂN TÁN.
    </p>
     <img src="https://i.ibb.co/xXjMKZS/Drawing5.jpg" height="70%" width="70%"> 
</h3>

<h4>
    <p style="color: blue">
        Dùng gói findSpark để khởi tạo môi trường chạy
    </p>
</h4>

In [1]:
import findspark
findspark.init()

<h4>
    <p style="color:red">
        Import các thư viện của PySpark
    </p>
</h4>

In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from os import listdir
from pyspark.sql.types import *
from pyspark.ml.evaluation import *

<h4>
    <p style="color:blue">
        Khai báo số memory mà mỗi Worker sẽ sử dụng để thực thi task
    </p>
</h4>

In [3]:
sc_conf = SparkConf()
sc_conf.setAppName("Sentiment Analysic Data mining Keras1")
sc_conf.setMaster("spark://192.168.1.94:7077")
sc_conf.set('spark.executor.memory', '7065m')
print(sc_conf.getAll())

[('spark.app.name', 'Sentiment Analysic Data mining Keras1'), ('spark.repl.local.jars', 'file:///C:/Users/practice/dist-spark-2.4.0-scala-2.11.8-all-0.9.0-dist/lib/bigdl-SPARK_2.4-0.9.0-jar-with-dependencies.jar'), ('spark.sql.catalogImplementation', 'in-memory'), ('spark.executor.extraClassPath', 'C:\\Users\\practice\\dist-spark-2.4.0-scala-2.11.8-all-0.9.0-dist\\lib\\bigdl-SPARK_2.4-0.9.0-jar-with-dependencies.jar'), ('spark.shuffle.reduceLocality.enabled', 'false'), ('spark.shuffle.blockTransferService', 'nio'), ('spark.files', 'file:///C:/Users/practice/dist-spark-2.4.0-scala-2.11.8-all-0.9.0-dist/lib/bigdl-0.9.0-python-api.zip'), ('spark.driver.extraClassPath', 'C:\\Users\\practice\\dist-spark-2.4.0-scala-2.11.8-all-0.9.0-dist\\lib\\bigdl-SPARK_2.4-0.9.0-jar-with-dependencies.jar'), ('spark.jars', 'file:///C:/Users/practice/dist-spark-2.4.0-scala-2.11.8-all-0.9.0-dist/lib/bigdl-SPARK_2.4-0.9.0-jar-with-dependencies.jar'), ('spark.speculation', 'false'), ('spark.master', 'spark://1

In [4]:
# tạo spark context, lưu ý nhớ thay đổi lại địa chỉ của spark cho phù hợp
sc = SparkContext.getOrCreate(sc_conf)
spark = SparkSession.builder.getOrCreate()
sqlContext = SQLContext(sc)

In [5]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [6]:
spark

In [7]:
# khai báo một số hằng số
NUMBER_INSTANCE_FOR_PC = 1000
NUMBER_FEATURE_VECTOR = 500

In [8]:
# đường dẫn đến tập train positive
TRAIN_PATH_POS = "./sentiment_data/aclImdb/train/pos/"
# đường dẫn đến tập train negative
TRAIN_PATH_NEG = "./sentiment_data/aclImdb/train/neg/"

In [9]:
# biến lưu dữ liệu train cả positive và negative
data_train = []

<h4>
    <p style="color:blue">
        Dữ liệu có 2 tập:
    </p>
    <p style="color:red">
        Positive: nhận xét tích cực, chứa các câu nhận xét mang ý nghĩa tích cực, sẽ được đánh label là 1
    </p>
    <p style="color:red">
        Negative: nhận xét tiêu cực, chứa các câu nhận xét mang ý nghĩa tiêu cực, sẽ được đánh label là 2
    </p>
</h4>

In [10]:
# đọc dữ liệu của data train positive
listfile_pos = listdir(TRAIN_PATH_POS)
listfile_pos = enumerate(listfile_pos)
listfile_pos

In [11]:
counter = 1
for index, file_pos in listfile_pos:
    try:
        with open(TRAIN_PATH_POS + file_pos, 'r') as f_pos:
            noidung = f_pos.read()
            noidung = noidung.lower()
            # đánh label cho tập data train positive là 1
            data_train.append((index, noidung, 1.0))
        if counter >= NUMBER_INSTANCE_FOR_PC:
            break
        else:
            counter = counter + 1
    except Exception as e:
        print(e)

'charmap' codec can't decode byte 0x9d in position 803: character maps to <undefined>


In [12]:
len(data_train)

1000

In [13]:
# lấy dữ liệu cho tập data train negative
listfile_neg = listdir(TRAIN_PATH_NEG)
listfile_neg = enumerate(listfile_neg)
listfile_neg

In [14]:
counter = 1
for index, file_neg in listfile_neg:
    try:
        with open(TRAIN_PATH_NEG + file_neg) as f_neg:
            noidung_neg = f_neg.read()
            noidung_neg = noidung_neg.lower()
            # đánh label cho data train negative là 0
            data_train.append((index, noidung_neg, 2.0))
            if counter >= NUMBER_INSTANCE_FOR_PC:
                break
            else:
                counter = counter + 1
    except Exception as e:
        print(e)

In [15]:
len(data_train)

2000

<h4>
    <p style="color:red">
        Tạo đối tượng DataFrame trong Spark để chứa dữ liệu train, đây là kiểu dữ liệu chuẩn trong việc xử lý phân tán và có thể thao tác dưới dạng SQL, lúc này dữ liệu train đã ở dạng phân tán khắp cluster
    </p>
</h4>

In [16]:
# tạo DataFrame trong Spark
sentenceDataFrame_train = spark.createDataFrame(data_train,
                                                ["id", "sentence", "label"])

<h4>
    <p style="color:blue">
        Tạo một cấu trúc Pipeline trong Spark gồm có:
    </p>
    <p style="color:green">
        1. Tokenizer để rời rạc hóa câu trong data train. Ví dụ: "I am bad" --> [ "I", "am", "bad" ]
    </p>
    <p style="color:green">
        2. StopWordsRemover để loại bỏ các từ không mang giá trị phân biệt, ví dụ: I, You,...
    </p>
    <p style="color:green">
        3. HashingTF để tính TF (Tern Frequency)
    </p>
    <p style="color:green">
        4. IDF để tính IDF, lúc này ứng với mỗi câu sẽ có một cột features là một vector đại diện cho dạng biểu diễn của câu đó
    </p>
    
</h4>

In [17]:
# Tokenize (rời rạc hóa) word trong câu in data train
tk_train = Tokenizer(inputCol="sentence", outputCol="words_raw")

# loại bỏ từ stop word ---> là những từ không mang ý nghĩa phân biệt nhiều,
# cũng như lập lại quá nhiều lần trong các câu, ví dụ: I, You,...
stop_remover_train = StopWordsRemover(inputCol="words_raw",
                                      outputCol="filtered")

# tính TF (Tern Frequency)
tf_train = HashingTF(inputCol="filtered",
                     outputCol="rawFeatures",
                     numFeatures=NUMBER_FEATURE_VECTOR)

# tính IDF (Invert Tern Frequency)
idf_train = IDF(inputCol="rawFeatures", outputCol="features")

In [18]:
tk_train

Tokenizer_4b731722c414

In [19]:
stop_remover_train

StopWordsRemover_ad448d27f387

In [20]:
tf_train

HashingTF_4d4803009191

In [21]:
idf_train

IDF_10847f2642d9

<h4>
    <p>
        Tạo đối tượng Pipeline gồm các bước như đã định nghĩa ở trên
    </p>
</h4>

In [22]:
pl = Pipeline(stages=[tk_train, stop_remover_train, tf_train, idf_train])

<h4>
    <p style="color:red">
        Bắt đầu Train tập dữ liệu ĐỂ LẤY FEATURE TF-IDF
    </p>
</h4>

In [23]:
modelF = pl.fit(sentenceDataFrame_train)

In [24]:
modelF

PipelineModel_10dfd8418f73

In [25]:
tf_data = modelF.transform(sentenceDataFrame_train)
tf_data

DataFrame[id: bigint, sentence: string, label: double, words_raw: array<string>, filtered: array<string>, rawFeatures: vector, features: vector]

In [26]:
tf_data_select = tf_data.select("features", "label")

<h4>
    <p style="color:red">
        SAU KHI ĐÃ CÓ ĐƯỢC VECTOR FEATURE VÀ LABEL CHO BỘ DỮ LIỆU TRAIN
    </p>
</h4>

In [27]:
tf_data_select

DataFrame[features: vector, label: double]

<h4>
    <p style="color:red">
        IMPORT THƯ VIỆN CỦA BIGDL
    </p>
</h4>

In [28]:
from bigdl.nn.layer import *
from bigdl.nn.criterion import *
from bigdl.optim.optimizer import *
from bigdl.util.common import *
from bigdl.dataset.transformer import *
from bigdl.dlframes.dl_classifier import *

<h4>
    <p>
        KHỞI TẠO MÔI TRƯỜNG CHO BIGDL
    </p>
</h4>

In [29]:
init_engine()

<h4>
    <p style="color:red">
        ĐỊNH NGHĨA MẠNG NEURAL KIỂU KERAS
    </p>
    <img src="https://pluralsight.imgix.net/course-images/keras-deep-learning-v1.png" height="70%" width="70%"/>
    <p style="color:green">
        Tham khảo thêm thông tin về các định nghĩa một Neural NetWork trong BigDL tại:
    </p>
    <a href="https://bigdl-project.github.io/0.9.0/#ProgrammingGuide/Model/Sequential/">https://bigdl-project.github.io/0.9.0/#ProgrammingGuide/Model/Sequential/</a>
</h4>

In [30]:
model = Sequential()
# Hidden layer with ReLu activation
model.add(Reshape([500]))
model.add(Linear(500, 1000).set_name('mlp_fc1'))
model.add(ReLU())
# Hidden layer with ReLu activation
model.add(Linear(1000, 500).set_name('mlp_fc2'))
model.add(ReLU())
# output layer
model.add(Linear(500, 2).set_name('mlp_fc3'))
model.add(LogSoftMax())

creating: createSequential
creating: createReshape
creating: createLinear
creating: createReLU
creating: createLinear
creating: createReLU
creating: createLinear
creating: createLogSoftMax


In [31]:
classNLL_criterion = ClassNLLCriterion()

creating: createClassNLLCriterion


<h4>
    <p style="color:blue">
        TẠO MODEL CLASSIFIER DỰA VÀO MẠNG NEURAL Ở TRÊN
    </p>
</h4>

In [32]:
classifier = DLClassifier(model=model,
                          criterion=classNLL_criterion,
                          feature_size=[500])

creating: createDLClassifier


<h4>
    <p style="color:red">
        SET CÁC THAM SỐ KHI TRAIN
    </p>
</h4>

In [33]:
classifier.setBatchSize(4).setMaxEpoch(10).setLearningRate(0.01)

DLClassifier_ccb835cac3a2

<h4>
    <p style="color:red">
        Bắt đầu Train MẠNG NEURAL trong BigDL
    </p>
</h4>

In [34]:
dlModel = classifier.fit(tf_data_select)

<h4>
    <p>
        In ra Model BigDL (Deep NN) sau khi train
    </p>
</h4>

In [174]:
dlModel

DLClassifierModel_1eb9f046b922

<h4>
    <p style="color:blue">
        Chuẩn bị tập dữ liệu test - đánh giá mạng Neural BigDL
    </p>
</h4>

In [178]:
# khai báo một số hằng số
NUMBER_INSTANCE_FOR_PC = 1000
# NUMBER_FEATURE_VECTOR = 500
# đường dẫn đến tập train positive
TEST_PATH_POS = "./semantic_data/aclImdb/test/pos/"
# đường dẫn đến tập train negative
TEST_PATH_NEG = "./semantic_data/aclImdb/test/neg/"
# biến lưu dữ liệu train cả positive và negative
data_test = []

In [179]:
# đọc dữ liệu của data train positive
listfile_pos = listdir(TEST_PATH_POS)
listfile_pos = enumerate(listfile_pos)
counter = 1
for index, file_pos in listfile_pos:
    try:
        with open(TEST_PATH_POS + file_pos, 'r') as f_pos:
            noidung = f_pos.read()
            noidung = noidung.lower()
            # đánh label cho tập data train positive là 1
            data_test.append((index, noidung, 1.0))
        if counter >= NUMBER_INSTANCE_FOR_PC:
            break
        else:
            counter = counter + 1
    except Exception as e:
        print(e)

'charmap' codec can't decode byte 0x9d in position 1899: character maps to <undefined>


In [180]:
# lấy dữ liệu cho tập data train negative
listfile_neg = listdir(TEST_PATH_NEG)
listfile_neg = enumerate(listfile_neg)
counter = 1
for index, file_neg in listfile_neg:
    try:
        with open(TEST_PATH_NEG + file_neg) as f_neg:
            noidung_neg = f_neg.read()
            noidung_neg = noidung_neg.lower()
            # đánh label cho data train negative là 2
            data_test.append((index, noidung_neg, 2.0))
            if counter >= NUMBER_INSTANCE_FOR_PC:
                break
            else:
                counter = counter + 1
    except Exception as e:
        print(e)

<h4>
    <p style="color:red">
        Tạo đối tượng DataFrame trong Spark để chứa dữ liệu train, đây là kiểu dữ liệu chuẩn trong việc xử lý phân tán và có thể thao tác dưới dạng SQL, lúc này dữ liệu train đã ở dạng phân tán khắp cluster
    </p>
</h4>

In [181]:
# tạo DataFrame trong Spark
sentenceDataFrame_test = spark.createDataFrame(data_test,
                                               ["id", "sentence", "label"])

In [182]:
sentenceDataFrame_test.show(100)

+---+--------------------+-----+
| id|            sentence|label|
+---+--------------------+-----+
|  0|i went and saw th...|  1.0|
|  1|actor turned dire...|  1.0|
|  2|as a recreational...|  1.0|
|  3|i saw this film i...|  1.0|
|  4|bill paxton has t...|  1.0|
|  5|i saw this film o...|  1.0|
|  6|maybe i'm reading...|  1.0|
|  7|i felt this film ...|  1.0|
|  8|this movie is ama...|  1.0|
|  9|"quitting" may be...|  1.0|
| 10|i loved this movi...|  1.0|
| 11|i was fortunate t...|  1.0|
| 12|i first saw this ...|  1.0|
| 13|i must say, every...|  1.0|
| 14|my wife is a ment...|  1.0|
| 15|i saw this film a...|  1.0|
| 16|"night of the hun...|  1.0|
| 17|even if you're a ...|  1.0|
| 18|i was surprised h...|  1.0|
| 19|i went into "nigh...|  1.0|
| 20|i have certainly ...|  1.0|
| 21|since this cartoo...|  1.0|
| 22|despite the title...|  1.0|
| 23|felix in hollywoo...|  1.0|
| 24|a gem of a cartoo...|  1.0|
| 25|this short is one...|  1.0|
| 26|felix is watching...|  1.0|
| 27|while

<h4>
    <p style="color:red">
      Dùng model TF-IDF đã train (ở trên) áp dụng (transform) lên dữ liệu test đã tạo để có được feature vector
    </p>
</h4>

In [183]:
df_data_test = modelF.transform(sentenceDataFrame_test)

In [184]:
df_data_test.show(100)

+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
| id|            sentence|label|           words_raw|            filtered|         rawFeatures|            features|
+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+
|  0|i went and saw th...|  1.0|[i, went, and, sa...|[went, saw, movie...|(500,[5,6,11,15,4...|(500,[5,6,11,15,4...|
|  1|actor turned dire...|  1.0|[actor, turned, d...|[actor, turned, d...|(500,[3,10,12,15,...|(500,[3,10,12,15,...|
|  2|as a recreational...|  1.0|[as, a, recreatio...|[recreational, go...|(500,[4,8,9,10,18...|(500,[4,8,9,10,18...|
|  3|i saw this film i...|  1.0|[i, saw, this, fi...|[saw, film, sneak...|(500,[3,9,25,44,4...|(500,[3,9,25,44,4...|
|  4|bill paxton has t...|  1.0|[bill, paxton, ha...|[bill, paxton, ta...|(500,[3,5,12,18,1...|(500,[3,5,12,18,1...|
|  5|i saw this film o...|  1.0|[i, saw, this, fi...|[saw, film,

<h4>
    <p style="color:green">
      Sử dụng model BigDL để predict data test
    </p>
</h4>

In [185]:
data_test_after_transform = dlModel.transform(df_data_test)

In [186]:
data_test_after_transform.show(100)

+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
| id|            sentence|label|           words_raw|            filtered|         rawFeatures|            features|prediction|
+---+--------------------+-----+--------------------+--------------------+--------------------+--------------------+----------+
|  0|i went and saw th...|  1.0|[i, went, and, sa...|[went, saw, movie...|(500,[5,6,11,15,4...|(500,[5,6,11,15,4...|       1.0|
|  1|actor turned dire...|  1.0|[actor, turned, d...|[actor, turned, d...|(500,[3,10,12,15,...|(500,[3,10,12,15,...|       1.0|
|  2|as a recreational...|  1.0|[as, a, recreatio...|[recreational, go...|(500,[4,8,9,10,18...|(500,[4,8,9,10,18...|       2.0|
|  3|i saw this film i...|  1.0|[i, saw, this, fi...|[saw, film, sneak...|(500,[3,9,25,44,4...|(500,[3,9,25,44,4...|       1.0|
|  4|bill paxton has t...|  1.0|[bill, paxton, ha...|[bill, paxton, ta...|(500,[3,5,12,18,1...|(500,[3,5

<h4>
    <p>
      Tạo đối tượng Evaluator có sẵn trong Spark MLLib, đánh giá model của BigDL
    </p>
</h4>

In [188]:
evaluator = MulticlassClassificationEvaluator()
result = evaluator.evaluate(data_test_after_transform)

<h4>
    <p style="color:red">
      In ra kết quả đánh giá là độ chính xác, có thể áp dụng nhiều thông số đánh giá khác. Tham khảo thêm tại:
    </p>
    <p>
        <a href="https://spark.apache.org/docs/latest/ml-classification-regression.html#multilayer-perceptron-classifier">https://spark.apache.org/docs/latest/ml-classification-regression.html#multilayer-perceptron-classifier</a>
    </p>
</h4>

In [189]:
result

0.6654196170629996

In [191]:
print("end_of_train_BigDL(Deep Neural Network Model) in Spark")
spark.stop()

end_of_train_BigDL(Deep Neural Network Model) in Spark
